In [1]:
import cv2 as cv
import pandas as pd
import random, os
import matplotlib.pyplot as plt
import numpy as np
import pyvista as pv
import miniball

In [21]:
test = './9/ros-projection-057184-055.png'
print(test.rsplit('-',2))
ros_id = test.rsplit('-',2)[1]
proj_id = test.rsplit('-',2)[2].split('.',2)[0]
print(ros_id, proj_id)

['./9/ros-projection', '057184', '055.png']
057184 055


In [12]:
# Load the JSON file
import json
params_path = '/glade/u/home/joko/ice3d/output/params_200_50.json'
with open(params_path, 'rb') as file:
    params = json.load(file)
params[75]

[[25.728020603569504,
  31.566005898898368,
  1.0559465112004924,
  0.9235238401447887,
  1.062846051511876,
  4],
 [0.9870573750545488,
  1.025943071786323,
  0.9113414199644319,
  1.1182592305508643,
  0.9669323822842779,
  0.9578224808743632,
  1.141544895825053,
  1.1106086484807067],
 [-0.1953923308982515,
  0.6997064796354285,
  -0.6871918795957528,
  0.348343425573144,
  0.8064503295756525,
  0.47780197130954655,
  -0.044999511612952896,
  -0.8384310002651846,
  0.5431468509978841,
  0.6665734794270578,
  -0.5996757346434333,
  -0.4427965783566583]]

In [2]:
base_folder = '/glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/projections'
ros_data = '/glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/data/ros-data-merged.txt'

In [3]:
# read single image
df_ros = pd.read_csv(ros_data)
df_ros.head()

,id,a,c,f_r0,f_hp,f_h0,n_arms,sa,vol,sa_eff,rho_eff
0,35000,13.817545,23.540122,0.867449,0.961225,0.943525,7,33247.371151,181231.947690,0.576030,0.138993
1,35001,13.817545,23.540122,0.867449,0.961225,0.943525,7,33022.961732,184536.198465,0.560433,0.137205
2,35002,13.817545,23.540122,0.867449,0.961225,0.943525,7,33530.598131,187678.178569,0.585745,0.145727
3,35003,13.817545,23.540122,0.867449,0.961225,0.943525,7,32592.037777,166089.393327,0.543729,0.120358
4,35004,13.817545,23.540122,0.867449,0.961225,0.943525,7,32892.678757,179534.751064,0.583751,0.142747


In [5]:
df_ros.dtypes

id           int64
a          float64
c          float64
f_r0       float64
f_hp       float64
f_h0       float64
n_arms       int64
sa         float64
vol        float64
sa_eff     float64
rho_eff    float64
dtype: object

In [10]:
df_ros[df_ros['id']==75]

,id,a,c,f_r0,f_hp,f_h0,n_arms,sa,vol,sa_eff,rho_eff
28123,75,25.728021,31.566006,1.055947,0.923524,1.062846,4,53832.477555,495428.865713,0.51989,0.158128


In [8]:
df_ros.shape

(70000, 11)

In [19]:
list(df_ros[df_ros.id==id].iloc[0])

[57184.0,
 24.71119830513499,
 33.377526155680094,
 1.1352751410568025,
 1.1970878710185386,
 1.1000915127090385,
 9.0,
 127929.02834462238,
 1148194.1098501142,
 0.6768917294241552,
 0.1486163739867864]

In [ ]:
# get random projection sample and plot
sample = df_ros.sample()
print(sample)
id = sample['id'].iloc[0]
n_arms = sample['n_arms'].iloc[0]
n_proj = random.randint(0, 99)
file_name = f'ros-projection-{id:06d}-{n_proj:03d}.png'
proj_dir = os.path.join(base_folder, f'{n_arms}')
file_path = os.path.join(proj_dir, file_name)
if os.path.exists(file_path):
    print(f'path exists: {file_path}')
else:
    print('path does not exist')
img = cv.imread(file_path)
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
def get_img(file_path):
    img = cv.imread(file_path)
    img = cv.cvtColor(img, cv.COLOR_BGR2YCR_CB)[...,0]
    return img

def show_img(im, figsize=None, ax=None, alpha=None):
    if not ax: fig,ax = plt.subplots(figsize=figsize)
    ax.imshow(im, alpha=alpha, cmap=plt.cm.gray)
    ax.set_axis_off()
    return ax

def get_border(image, width):
    bg = np.zeros(image.shape)
    contours, _ = cv.findContours(image.copy(), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    biggest = 0
    bigcontour = None
    for contour in contours:
        area = cv.contourArea(contour) 
        if area > biggest:
            biggest = area
            bigcontour = contour
    return cv.drawContours(bg, [bigcontour], 0, (255, 255, 255), width).astype(bool), contours 

def get_aspect_ratio(cnt):
    rect = cv.minAreaRect(cnt)
    # get length and width of contour
    x = rect[1][0]
    y = rect[1][1]
    rect_length = max(x, y)
    rect_width = min(x, y)
    phi = rect_width / rect_length
    return phi

def get_aspect_ratio_elip(cnt):
    ellipse = cv.fitEllipse(cnt)
    # Get width and height of rotated ellipse
    widthE = ellipse[1][0]
    heightE = ellipse[1][1]
    if widthE > heightE:
        phiE = heightE / widthE
    else:
        phiE = widthE / heightE
    return phiE

def get_extreme_pts(cnt):
    left = tuple(cnt[cnt[:, :, 0].argmin()][0])
    right = tuple(cnt[cnt[:, :, 0].argmax()][0])
    top = tuple(cnt[cnt[:, :, 1].argmin()][0])
    bottom = tuple(cnt[cnt[:, :, 1].argmax()][0])
    extreme_pts = np.std([left, right, top, bottom])
    return extreme_pts

def get_contour_area(cnt):
    area = cv.contourArea(cnt)
    return area

def get_contour_perimeter(cnt):
    perimeter = cv.arcLength(cnt, True)
    return perimeter

def get_min_circle(cnt):
    center ,radius = cv.minEnclosingCircle(cnt)
    perimeter_circle = 2*np.pi*radius
    area_circle = np.pi*(radius**2)
    return center, radius, perimeter_circle, area_circle

def get_area_ratio(cnt):
    area = get_contour_area(cnt)
    _,_,_,area_circle = get_min_circle(cnt)
    area_ratio = area/area_circle
    return area_ratio

def get_complexity(cnt):
    _, radius, _, _ = get_min_circle(cnt)
    area = get_contour_area(cnt)
    perimeter = get_contour_perimeter(cnt)
    Ac = np.pi * radius ** 2
    complexity = 10*(0.1-(area / (np.sqrt(area / Ac) * perimeter ** 2)))
    return complexity

def get_circularity(cnt):
    area = get_contour_area(cnt)
    perimeter = get_contour_perimeter(cnt)
    circularity = 4*np.pi*(area/(perimeter**2))
    return circularity

In [ ]:
def process_img(path):
    img = get_img(path)
    _, contours = get_border(img, 5)
    cnt = contours[0]
    aspect_ratio = get_aspect_ratio(cnt)
    aspect_ratio_elip = get_aspect_ratio_elip(cnt)
    extreme_pts = get_extreme_pts(cnt)
    contour_area = get_contour_area(cnt)
    contour_perimeter = get_contour_perimeter(cnt)
    area_ratio = get_area_ratio(cnt)
    complexity = get_complexity(cnt)
    circularity = get_circularity(cnt)
    img_features = [aspect_ratio, aspect_ratio_elip, extreme_pts, \
        contour_area, contour_perimeter, area_ratio, \
        complexity, circularity]
    return img_features
img_features = process_img(file_path)
img_features

In [ ]:
%%time
# time estimate 
n = 100
for i in range(n):
    sample = df_ros.sample()
    id = sample['id'].iloc[0]
    n_arms = sample['n_arms'].iloc[0]
    n_proj = random.randint(0, 99)
    file_name = f'ros-projection-{id:06d}-{n_proj:03d}.png'
    proj_dir = os.path.join(base_folder, f'{n_arms}')
    file_path = os.path.join(proj_dir, file_name)
    img_features = process_img(file_path)

- ~2-3 seconds per 100 images
- on single core, would take ~58 hrs to process 7 million images
- on 100 cores, would take ~35 minutes

In [ ]:
sample

In [ ]:
max_rho = df_ros.loc[df_ros['rho_eff'].idxmax()]
max_rho

In [ ]:
# plot rosette with max rho_eff
sample = max_rho
id = int(sample['id'])
n_arms = int(sample['n_arms'])
n_proj = random.randint(0, 99)
file_name = f'ros-projection-{id:06d}-{n_proj:03d}.png'
proj_dir = os.path.join(base_folder, f'{n_arms}')
file_path = os.path.join(proj_dir, file_name)
if os.path.exists(file_path):
    print(f'path exists: {file_path}')
else:
    print('path does not exist')
img = cv.imread(file_path)
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
# calculate rho_eff with pyvista
stl_base = '/glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl'
stl_dir = os.path.join(stl_base, f'{n_arms}')
file_name = f'ros-test-{id:06d}.stl'
stl_path = os.path.join(stl_dir, file_name)
if os.path.exists(stl_path):
    print(f'path exists: {stl_path}')
else:
    print('path does not exist')

In [ ]:
# load mesh
mesh = pv.read(stl_path)

# import points and get center and radius of bounding sphere
mesh_points = np.asarray(mesh.points)
c, r2 = miniball.get_bounding_ball(mesh_points)
r = np.sqrt(r2) # r2 = radius squared, r = radius

# prepare plot
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# plot points
ax.scatter(mesh_points[:,0], mesh_points[:,1], mesh_points[:,2])

# plot sphere
u, v = np.mgrid[0:2*np.pi:50j, 0:np.pi:50j]
x = r*np.cos(u)*np.sin(v)
y = r*np.sin(u)*np.sin(v)
z = r*np.cos(v)
ax.plot_surface(x+c[0], y+c[1], z+c[2], color='r', alpha=0.2)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_box_aspect(aspect=None, zoom=0.8)
plt.show()
print(c, r)

In [ ]:
print(mesh.volume)

In [ ]:
print(r)

In [ ]:
v_sphere = (4/3)*np.pi*(r**3)
rho = mesh.volume/v_sphere
rho

!!! IMPORTANT !!!
- there was a prior bug in previous code that calculated volume of enclosing sphere as 'v_sphere = np.pi*(r**2)'
- This is why the effective density was so much higher than the correct effective densities calculated
- See in "regression_asym_create_data.ipynb" notebook in ice3d-old directory on xcite

In [ ]:
df_ros.rho_eff.hist(bins=100)

In [ ]:
log_rho = np.log(df_ros.rho_eff)
log_rho.hist(bins=100)

In [ ]:
df_ros.sa_eff.hist(bins=100)

In [ ]:
log_sa = np.log(df_ros.sa_eff)
log_sa.hist(bins=100)

# Generate png list using linux command 

It's too slow to generate path list using python. Use linux commands instead:  
```find . -type f -name "*.png" > img_relative_paths.txt```

In [4]:
import os 
projections_dir = '/glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/projections'
img_paths_txt = '/glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/projections/img_relative_paths.txt'
with open(img_paths_txt, 'r') as file:
    rel_paths = [line.strip().replace('./','') for line in file]
    img_paths = [os.path.join(projections_dir, i) for i in rel_paths]
img_paths[:10]

['/glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/projections/phips/6/ros-projection-022282-000-phips.png',
 '/glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/projections/phips/6/ros-projection-020298-001-phips.png',
 '/glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/projections/phips/6/ros-projection-029536-001-phips.png',
 '/glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/projections/phips/6/ros-projection-020298-004-phips.png',
 '/glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/projections/phips/6/ros-projection-022415-004-phips.png',
 '/glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/projections/phips/6/ros-projection-027327-001-phips.png',
 '/glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/projections/phips/6/ros-projection-024274-005-phips.png',
 '/glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/projections/phips/6/ros-projection-022282-007-phips.png',
 '/glade/derecho/scratch/joko/sy

In [5]:
len(img_paths)

21000000

In [3]:
test_path = '/glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/projections/phips/6/ros-projection-022282-000-phips.png'
filename = test_path.rsplit('/',1)[1]
filename.split('-')
ros_id = filename.split('-')[2]
proj_id = filename.split('-')[3]
unique_id = ros_id + '_' + proj_id
view = filename.split('-')[-1].split('.')[0]
print(f'ros_id: {ros_id}')
print(f'proj_id: {proj_id}')
print(f'unique_id: {unique_id}')
print(f'view: {view}')

ros_id: 022282
proj_id: 000
unique_id: 022282_000
view: phips


In [ ]:
# %%time
# from pathlib import Path

# directory = "/glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/projections"
# file_paths = [str(file) for file in Path(directory).rglob('*') if file.is_file()]

# print(file_paths)
